In [21]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

In [22]:
column_names = list(range(0,4))
column_names.append('Classification')

df = pd.read_csv("banknote_authentication.data", names = column_names) 
df

,0,1,2,3,Classification
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [23]:
df.dtypes

0                 float64
1                 float64
2                 float64
3                 float64
Classification      int64
dtype: object

In [24]:
#find number of null entries to see if there is a need to clean the dataset
df.isnull().sum()
#df = df.dropna()

0                 0
1                 0
2                 0
3                 0
Classification    0
dtype: int64

In [25]:
df.nunique()

0                 1338
1                 1256
2                 1270
3                 1156
Classification       2
dtype: int64

In [26]:
X = df.drop('Classification', axis=1)
y = df['Classification']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [28]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tf.random.set_seed(42)
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(128, activation='relu', input_shape = (4,)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy",
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state =42, shuffle=True)

sum_acc_score = 0
sum_prec_score = 0
sum_recall_score = 0
sum_f1_score = 0


for train_index, test_index in kf.split(X):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()

    X_test = pd.DataFrame()
    y_test = pd.DataFrame()
    
    for index in train_index:
        X_train = X_train.append(X.iloc[index])
        y_train = y_train.append(pd.Series(y.iloc[index]), ignore_index = True)
        
    for index in test_index:
        X_test = X_test.append(X.iloc[index])
        y_test = y_test.append(pd.Series(y.iloc[index]), ignore_index = True)

    train_X = tf.convert_to_tensor(X_train)
    test_X = tf.convert_to_tensor(X_test)

    #one hot encode y_train and y_test
    train_y = tf.keras.utils.to_categorical(y_train)
    test_y = tf.keras.utils.to_categorical(y_test)

            
    history = model.fit(train_X, train_y, epochs=50, validation_data = (test_X, test_y),callbacks=[callback])
    predictions = model.predict(test_X)
    
    predicted_classes = predictions.argmax(axis=1)
    test_y_classes = test_y.argmax(axis=1)
    
    sum_acc_score += accuracy_score(test_y_classes, predicted_classes)
    sum_prec_score += precision_score(test_y_classes, predicted_classes, average = 'micro')
    sum_recall_score += recall_score(test_y_classes, predicted_classes, average = 'micro')
    sum_f1_score += f1_score(test_y_classes, predicted_classes, average = 'micro')

Epoch 1/50
39/39 [==============================] - 1s 15ms/step - loss: 0.2194 - accuracy: 0.9360 - precision: 0.9360 - recall: 0.9360 - val_loss: 0.0554 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/50
39/39 [==============================] - 0s 3ms/step - loss: 0.0300 - accuracy: 0.9976 - precision: 0.9976 - recall: 0.9976 - val_loss: 0.0144 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/50
39/39 [==============================] - 0s 4ms/step - loss: 0.0102 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0055 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/50
39/39 [==============================] - 0s 3ms/step - loss: 0.0053 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0034 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/50
39/39 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000 - precision: 

Epoch 37/50
39/39 [==============================] - 0s 2ms/step - loss: 4.1252e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 2.0713e-05 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 38/50
39/39 [==============================] - 0s 3ms/step - loss: 3.8548e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.9186e-05 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 39/50
39/39 [==============================] - 0s 3ms/step - loss: 3.6440e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.7783e-05 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 40/50
39/39 [==============================] - 0s 3ms/step - loss: 3.4299e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.7002e-05 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 41/50
39/39 [==============================] - 0s 3ms/step - loss: 3.2

39/39 [==============================] - 0s 2ms/step - loss: 7.8129e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 4.7061e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 23/50
39/39 [==============================] - 0s 3ms/step - loss: 7.5128e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 4.4093e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 24/50
39/39 [==============================] - 0s 3ms/step - loss: 7.2483e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 4.3167e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 25/50
39/39 [==============================] - 0s 3ms/step - loss: 6.9839e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 4.1462e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 26/50
39/39 [==============================] - 0s 3ms/step - loss: 6.7130e-06 - ac

39/39 [==============================] - 0s 3ms/step - loss: 2.0535e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 5.3712e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 8/50
39/39 [==============================] - 0s 3ms/step - loss: 1.9883e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 5.2597e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 9/50
39/39 [==============================] - 0s 3ms/step - loss: 1.9325e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 5.2095e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 10/50
39/39 [==============================] - 0s 3ms/step - loss: 1.8790e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 5.1339e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 11/50
39/39 [==============================] - 0s 2ms/step - loss: 1.8268e-06 - accu

39/39 [==============================] - 0s 3ms/step - loss: 8.1054e-07 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.0938e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 43/50
39/39 [==============================] - 0s 3ms/step - loss: 7.8846e-07 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.0265e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 44/50
39/39 [==============================] - 0s 4ms/step - loss: 7.7054e-07 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 2.9820e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 45/50
39/39 [==============================] - 0s 4ms/step - loss: 7.5149e-07 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 2.9509e-06 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 46/50
39/39 [==============================] - 0s 4ms/step - loss: 7.3147e-07 - ac

...

39/39 [==============================] - 0s 3ms/step - loss: 9.7279e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 5.3643e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/50
39/39 [==============================] - 0s 3ms/step - loss: 9.5373e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 5.2920e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/50
39/39 [==============================] - 0s 4ms/step - loss: 9.3480e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 5.2476e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/50
39/39 [==============================] - 0s 3ms/step - loss: 9.1458e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 5.2281e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 6/50
39/39 [==============================] - 0s 4ms/step - loss: 8.9919e-09 - accura

39/39 [==============================] - 0s 4ms/step - loss: 5.0633e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 2.9544e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 38/50
39/39 [==============================] - 0s 4ms/step - loss: 4.9695e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 2.9431e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 39/50
39/39 [==============================] - 0s 4ms/step - loss: 4.8954e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 2.8423e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 40/50
39/39 [==============================] - 0s 4ms/step - loss: 4.8133e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 2.8224e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 41/50
39/39 [==============================] - 0s 4ms/step - loss: 4.7232e-09 - ac

39/39 [==============================] - 0s 4ms/step - loss: 2.7433e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.1641e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 23/50
39/39 [==============================] - 0s 3ms/step - loss: 2.6949e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.0903e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 24/50
39/39 [==============================] - 0s 4ms/step - loss: 2.6697e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.0584e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 25/50
39/39 [==============================] - 0s 4ms/step - loss: 2.6167e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 2.9722e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 26/50
39/39 [==============================] - 0s 4ms/step - loss: 2.5816e-09 - ac

39/39 [==============================] - 0s 4ms/step - loss: 1.7717e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.1454e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 8/50
39/39 [==============================] - 0s 3ms/step - loss: 1.7520e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.1386e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 9/50
39/39 [==============================] - 0s 4ms/step - loss: 1.7248e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.1197e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 10/50
39/39 [==============================] - 0s 4ms/step - loss: 1.7034e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 1.1054e-09 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 11/50
39/39 [==============================] - 0s 3ms/step - loss: 1.6752e-09 - accu

39/39 [==============================] - 0s 4ms/step - loss: 1.1042e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 7.1430e-10 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 43/50
39/39 [==============================] - 0s 4ms/step - loss: 1.0795e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 7.0255e-10 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 44/50
39/39 [==============================] - 0s 4ms/step - loss: 1.0696e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 6.8722e-10 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 45/50
39/39 [==============================] - 0s 3ms/step - loss: 1.0601e-09 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 6.8383e-10 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 46/50
39/39 [==============================] - 0s 4ms/step - loss: 1.0427e-09 - ac

In [29]:
print('Average Accuracy Score: ', sum_acc_score/10)
print('Average Precision Score: ', sum_prec_score/10)
print('Average Recall Score: ', sum_recall_score/10)
print('Average F1-Score: ', sum_f1_score/10)



Average Accuracy Score:  1.0
Average Precision Score:  1.0
Average Recall Score:  1.0
Average F1-Score:  1.0
